In [1]:
%matplotlib inline
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
from keras import metrics

from vgg16_avg import VGG16_Avg

ModuleNotFoundError: No module named 'keras'

In [2]:
#Gets all images

path = '/home/ubuntu/StyleGAN/train/'
all_images = glob.glob(path+'*/*'); len(all_images)

NameError: name 'glob' is not defined

In [3]:
file = all_images[1707] #taken from imagenet (is the truck photo)
img = Image.open(file)
img


NameError: name 'all_images' is not defined

In [4]:
sty_dir = '/home/ubuntu/StyleGAN/starry_night.jpg' #style photo
style = Image.open(sty_dir).resize(img.size, Image.ANTIALIAS); style

NameError: name 'Image' is not defined

## VGG Preprocess and Imagenet Mean subtractions

rgb_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
preproc = lambda x : (x - rgb_mean)[:, :, :, ::-1]
deproc = lambda x, s : np.clip(x.reshape(s)[:, :, :, ::-1] + rgb_mean, 0, 255)
img_arr = preproc(np.expand_dims(np.array(img), 0))
style_arr = preproc(np.expand_dims(np.array(style), 0))
img_shape = img_arr.shape
sty_shape = style_arr.shape
img_shape, sty_shape

### Developing model


In [6]:
input_shape = img_arr.shape[1:] #expanding dims for keras purposes (only takes 4D tensor)

NameError: name 'img_arr' is not defined

In [7]:
model = VGG16_Avg(include_top=False, input_shape=input_shape)

NameError: name 'VGG16_Avg' is not defined

In [8]:
cont_output = model.get_layer('block5_conv1').output; cont_output #modify output convolution block to vary results

NameError: name 'model' is not defined

In [9]:
from keras.models import Model

ModuleNotFoundError: No module named 'keras'

In [10]:
cont_model = Model(model.input, cont_output)
targ = K.variable(cont_model.predict(img_arr)) #equivalent to tf.variable as tf is backend

NameError: name 'Model' is not defined

In [11]:
class Evaluator(object): #separating loss and gradient values
    def __init__(self, func, img_shape): self.func, self.img_shape = func, img_shape
        
    def loss(self, x):
        loss_, self.grad_matrix = self.func([x.reshape(img_shape)])
        return loss_.astype(np.float64)
    
    def grads(self, x): return self.grad_matrix.flatten().astype(np.float64)

In [12]:
loss = K.mean(metrics.mse(cont_output, targ))
grads = K.gradients(loss, model.input)
fn = K.function([model.input], [loss] + grads)
eval_obj = Evaluator(fn, img_shape)

NameError: name 'K' is not defined

In [13]:
img_prog_path = '/home/ubuntu/StyleGAN/results/'

In [14]:
def recreate_content(eval_obj, niter, x):
    for i in range(niter):
        x, min_val, info = fmin_l_bfgs_b(eval_obj.loss, x.flatten(), fprime=eval_obj.grads, maxfun=20)
        x = np.clip(x, -127, 127) 
        
        print('Current loss value:', min_val)
        imsave(f'{img_prog_path}res_at_iteration_{i}.png', deproc(x.copy(), img_shape)[0])
    return x

In [15]:
rand_img = lambda shape: np.random.uniform(0, 255, shape) #creating noise as initial input
x = rand_img(img_shape)
plt.imshow(x[0])

NameError: name 'img_shape' is not defined

In [16]:
iters = 10
x = recreate_content(eval_obj, iters, x)

NameError: name 'eval_obj' is not defined

In [17]:
Image.open(img_prog_path+'res_at_iteration_9.png') #Display

NameError: name 'Image' is not defined

## Style Recreate

In [18]:
def plot_arr(array): plt.imshow(deproc(array, array.shape)[0].astype('uint8'))
    

In [19]:
total_conv_blocks = 5 #vgg case
output_dict = {layer.name : layer.output for layer in model.layers}
model_outs = [output_dict['block{}_conv1'.format(o)] for o in range(1, total_conv_blocks + 1)] #all possible conv blocks 

NameError: name 'model' is not defined

In [20]:
sty_model = Model(model.inputs, model_outs)
targs = [K.variable(o) for o in sty_model.predict(style_arr)] #output of model

NameError: name 'Model' is not defined

In [21]:
def gram_matrix(orig_mat): #gramian matrix 
    features = K.batch_flatten(K.permute_dimensions(orig_mat, (2, 0, 1)))
    return K.dot(features, K.transpose(features)) / orig_mat.get_shape().num_elements()
def style_loss(x, targs): return metrics.mse(gram_matrix(x), gram_matrix(targs))

In [22]:
loss = sum(K.mean(style_loss(l1[0], l2[0])) for l1, l2 in zip(model_outs, targs))
grads = K.gradients(loss, model.input)
sty_fn = K.function([model.input], [loss] + grads)
eval_obj = Evaluator(sty_fn, sty_shape)

NameError: name 'model_outs' is not defined

In [24]:
rand_img = lambda shape: np.random.uniform(0, 255, shape)
x = rand_img(sty_shape)
x = scipy.ndimage.filters.gaussian_filter(x, [0,2,2,0]) #gaussian filters seem to do better than a uniform distribution

NameError: name 'sty_shape' is not defined